In [22]:
from bs4 import BeautifulSoup
import psycopg2 as pg2
import logging
import requests
import time
import csv
with open('rsites.csv', 'rb') as csvfile:
    u = list(csv.reader(csvfile))

logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%a, %d %b %Y %H:%M:%S',
                    filename='log1.log',
                    filemode='w')

In [18]:
for x in u[0:3]:
    print x[0], x[1],x[2]

National Gallery /Attraction_Review-g186338-d188862-Reviews-National_Gallery-London_England.html 18896
British Museum /Attraction_Review-g186338-d187555-Reviews-British_Museum-London_England.html 35510
V&A - Victoria and Albert Museum /Attraction_Review-g186338-d187556-Reviews-V_A_Victoria_and_Albert_Museum-London_England.html 20440


In [27]:
for x in u[0:3]:
    print int(x[2])/10+1

1890
3552
2045


In [53]:
for x in u[868:870]:
    for i in range(10,int(x[2])+1,10):

        url = 'https://www.tripadvisor.com' + x[1].replace('-Reviews-','-Reviews-or{}-'.format(i))

        print url

https://www.tripadvisor.com/Attraction_Review-g186338-d548740-Reviews-or10-Queen_Mother_Sports_Centre-London_England.html
https://www.tripadvisor.com/Attraction_Review-g186338-d548740-Reviews-or20-Queen_Mother_Sports_Centre-London_England.html
https://www.tripadvisor.com/Attraction_Review-g186338-d10352332-Reviews-or10-Excess_Baggage_Company-London_England.html


In [60]:
page = 0
for x in u[868:870]:
    for i in range(10,int(x[2])+1,10):
        page = page+1
        try:
            url = 'https://www.tripadvisor.com' + x[1].replace('-Reviews-','-Reviews-or{}-'.format(i))
            page_recq = requests.get(url, timeout=5)
            logging.info('%s', url)
            print page
        except:
            logging.exception("Fail to requestion link: %", x[1])
        soup = BeautifulSoup(page_recq.content, "html.parser")
#=====================================================================================================================
# Reviews
#=====================================================================================================================
        r0 = soup.find_all('div', {'class':'reviewSelector'})
        for r in r0:
#=====================================================================================================================
# User name
#=====================================================================================================================
            try:
                user = r.find('div', {'class':'username mo'}).getText()
            except:
                logging.warning("Fail to aquire userid: site: %s", x[0])
                continue

#=====================================================================================================================
# User contributions and up votes
#=====================================================================================================================
            try:
                cu = r.find('div', {'class':'memberBadgingNoText'})
                cuv = cu.find_all('span', {'class':'badgetext'})
                user_Contributions = cuv[0].getText()
            except:
                user_Contributions = 0
                user_UpVotes = 0
            try:
                user_UpVotes = cuv[1].getText()
            except:
                user_UpVotes = 0
#=====================================================================================================================
# User location
#=====================================================================================================================
            try:
                user_location = r.find('div', {'class':'location'}).getText()
            except:
                user_location = 'missing'
#=====================================================================================================================
# Rating
#=====================================================================================================================
            try:
                user_rating = r.find('div', {'class':'ratingInfo'}).next['class'][1][-2:]
            except:
                user_rating = 'missing'
#=====================================================================================================================
# Review date
#=====================================================================================================================
            try:
                review_date = r.find('span', {'class':'ratingDate relativeDate'})['title']
            except:
                review_date = 'missing'
#=====================================================================================================================
# Mobile?
#=====================================================================================================================
            try:
                viamobile = r.find('span', {'class':'viaMobile'}).getText()
            except:
                viamobile = 'no'
#=====================================================================================================================
# Review quote
#=====================================================================================================================
            try:
                quote = r.find('span', {'class':'noQuotes'}).getText()
            except:
                quote = 'missing'

            try:
                review = r.find('p', {'class':'partial_entry'}).getText()
            except:
                review = 'missing'

            try:
                rlink = r.find('span', {'class':'noQuotes'}).previous['href']
            except:
                rlink = 'missing'
#=====================================================================================================================
# Save to database
#=====================================================================================================================
            try:
                conn = pg2.connect(host="localhost",database="suppliers", user="postgres", password="postgres",connect_timeout=10)
                cur = conn.cursor()
                sql = """INSERT INTO r0(userID, attraction, user_location, user_Contributions, user_UpVotes, viaMobile, review_quote, review_content, reviewlink) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s);"""
                cur.execute(sql, (user, x[0], user_location, user_Contributions, user_UpVotes, viamobile, quote, review, rlink))
                conn.commit()
            except:
                logging.exception('Data base (revi) error on page : %s', url)


0
0
0


In [57]:
conn = pg2.connect(host="localhost",database="suppliers", user="postgres", password="postgres",connect_timeout=10)
cur = conn.cursor()

In [58]:
sql = """INSERT INTO r0(userID, attraction, user_location, user_Contributions, user_UpVotes, viaMobile, review_quote, review_content, reviewlink) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s);"""
cur.execute(sql, (user, attraction, user_location, user_Contributions, user_UpVotes, viamobile, quote, review, rlink))
conn.commit()

NameError: name 'attraction' is not defined

In [15]:
('-Reviews-or%d-',10)

('-Reviews-or%d-', 10)

In [47]:
for x in u[870:8704]:
    print x

['Jurassic Kingdom', '/Attraction_Review-g186338-d12340074-Reviews-Jurassic_Kingdom-London_England.html', '36']


In [71]:
r.find('div', {'class':'ratingInfo'}).next['class'][1][-2:]

u'10'

In [70]:
bubble[-2:]

u'10'